<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 9: InSAR Time Series Analysis using GIAnT within Jupyter Notebooks <font color='rgba(200,0,0,0.2)'> -- [## Points] </font> </b> </font>

<br>
<font size="4"> <b> Franz J Meyer & Joshua J C Knicely; University of Alaska Fairbanks</b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /><font color='rgba(200,0,0,0.2)'> <b>Due Date: </b>NONE</font>
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. The primary goal of this lab is to demonstrate how to process InSAR data, specifically interferograms, using the Generic InSAR Analysis Toolbox (<a href="http://earthdef.caltech.edu/projects/giant/wiki" target="_blank">GIAnT</a>) in the framework of *Jupyter Notebooks*.<br>

<b>Our specific objectives for this lab are to:</b>

- Learn how to download data from a Hyp subscription using the ASF tools. 
- Learn how to prepare data for GIAnT. 
- Use GIAnT to create maps of surface deformation. 
    -  Understand its capabilities. 
    -  Understand its limitations. 
</font>

<br>
<font face="Calibri">

<font size="5"> <b> Target Description </b> </font>

<font size="3"> In this lab, we will analyse the volcano Sierra Negra. This is a highly active volcano on the Galapagos hotpsot. The most recent eruption occurred from 29 June to 23 August 2018. The previous eruption occurred in October 2005, prior to the launch of the Sentinel-1 satellites, which will be the source of data we use for this lab. We will be looking at the deformation that occurred prior to the volcano's 2018 eruption. </font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES NO HOMEWORK ASSIGNMENTS.</b></font> <br>

Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

<font face='Calibri'><font size='5'><b>Overview</b></font>
<br>
<font size='3'><b>About GIAnT</b>
<br>
GIAnT is a Python framework that allows rapid time series analysis of low amplitude deformation signals. It allows users to use multiple time series analysis technqiues: Small Baseline Subset (SBAS), New Small Baseline Subset (N-SBAS), and Multiscale InSAR Time-Series (MInTS). As a part of this, it includes the ability to correct for atmospheric delays by assuming a spatially uniform stratified atmosphere. 
<br><br>
<b>Limitations</b>
<br>
GIAnT has a number of limitations that are important to keep in mind as these can affect its effectiveness for certain applications. It implements the simplest time-series inversion methods. Its single coherence threshold is very conservative in terms of pixel selection. It does not include any consistency checks for unwrapping errors. It has a limited dictionary of temporal model functions. It cannot correct for atmospheric effects due to differing surface elevations. 
<br><br>
<b>Steps to use GIAnT</b><br>
Although GIAnT is an incredibly powerful tool, it requires very specific input. Because of the input requirements, the bulk of the lab and code below is dedicated to getting our data into a form that GIAnT can manipulate and to creating files that tell GIAnT what to do. The general steps of this lab to use GIAnT are below. 

- Download Data
- Identify Area of Interest
- Subset (Crop) Data to Area of Interest
- Prepare Data for GIAnT
    - Adjust file names
    - Remove potentially disruptive default values (optional)
    - Convert data from '.tiff' to '.flt' format
- Create Input Files for GIAnT
    - Create 'ifg.list'
    - Create 'date.mli.par'
    - Make prepxml_SBAS.py
    - Run prepxml_SBAS.py
    - Make userfn.py
- Run GIAnT
    - PrepIgramStack.py
    - ProcessStack.py
    - SBASInvert.py
    - SBASxval.py
- Data Visualization

<br><br>
More information about GIAnT can be found here: (<a href="http://earthdef.caltech.edu/projects/giant/wiki" target="_blank">http://earthdef.caltech.edu/projects/giant/wiki</a>).

<font face='Calibri'><font size='5'><b>1. Import Python Libraries</b></font><br>
    <font size='3'>Let's import the Python libraries we will need to run this lab. </font></font>

In [ ]:
import gdal, os, osr, h5py, shutil, re, sys
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from getpass import getpass
from asf_hyp3 import API
from IPython.display import HTML
from matplotlib import animation, rc
from datetime import date
import zipfile
import datetime # for date
import glob

from asf_notebook import download_hyp3_products
from asf_notebook import new_directory
from asf_notebook import earthdata_login
from asf_notebook import remove_nan_subsets

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Download Data from Hyp3 Subscription </b> </font>

<font size="3"> We will begin by acquiring the interferograms of Sierra Negra from our Hyp3 subscription. <br> <i> THIS MAY NEED TO BE ALTERED. HOW WILL STUDENTS DL FROM HYP??? </i> <br> The first cell below acquires user credentials and creates a .netrc file in order to access the Hyp3 subscription. </font>
<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. Setup an EarthData account if you do not yet have one. <font color='rgba(200,0,0,0.2)'><b>Note that EarthData's ULA applies when accessing the Hyp3 API from this notebook. If you have not acknowleged the ULA in EarthData, you will need to navigate to EarthData's home page and complete that process.</b></font>
<br><br>
<b>Login to Earthdata:</b> </font>

In [ ]:
api = earthdata_login()

<font face="Calibri" size="3"> Before we download anything, let's <b>first create a working directory for this analysis and change into it:</b> </font>

In [ ]:
path = "/home/jovyan/notebooks/ASF/GEOS_657_Labs/lab_9_Hyp3_data"
new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3"><b>Create a folder in which to download your interferograms:</b> </font>

In [ ]:
new_directory("ingrams")
products_path = f"{path}/ingrams"

<font face="Calibri" size="3"><b>Set a date range, flight direction, and path of products to download:</b> </font>

In [ ]:
date_range = [datetime.date(2017, 2, 1), datetime.date(2017, 10, 20)]

########## NOTE: Currently filtering by path and flight_direction doesn't work for InSAR products #########


# uncomment code below to download all products
date_range = [None, None]
direction = None

<font face="Calibri" size="3"><b>Download the products:</b> </font>

In [ ]:
subscription_id = download_hyp3_products(
    api, products_path)#, start_date=date_range[0], end_date=date_range[1], flight_direction=direction)

<font face="Calibri" size="3"><b>Grab the paths of the amplitude, interferogram, and coherence imagery:</b> </font>

In [ ]:
amp_paths = !ls ingrams/*/*_amp.tif | sort
print(f"amp_paths[0]: {amp_paths[0]}")
ingram_paths = !ls ingrams/*/*_unw_phase.tif | sort
print(f"ingram_paths[0]: {ingram_paths[0]}")
cohr_paths = !ls ingrams/*/*_corr.tif | sort
print(f"cohr_paths[0]: {cohr_paths[0]}")

<font face='Calibri' size='3'><b>Write a function to sort through the amplitude, interferogram, and coherence paths, removing those that occur either before or after the eruption:</b></font>

In [ ]:
# Function to return entries of '*_paths' that occur before or after a desired date. 
def sortDates(datesToSort, condition, eruptionDate):
    bad = []
    if condition.lower() in 'before':
        # add to list 'bad' if either date is after the the eruptionDate (i.e., before our desired timeframe). 
        for i in range(len(datesToSort)):
            mDate, sDate = datesToSort[i][13:21], datesToSort[i][29:37]
            if (int(mDate) > eruptionDate) or (int(sDate) > eruptionDate):
                bad.append(int(i))
    elif condition.lower() in 'after':
        # add to list 'bad' if either date is before the the eruptionDate (i.e., after our desired timeframe). 
        for i in range(len(datesToSort)):
            mDate, sDate = datesToSort[i][13:21], datesToSort[i][29:37]
            if (int(mDate) < eruptionDate) or (int(sDate) < eruptionDate):
                bad.append(int(i))
    else:
        print(f'Input \'condition\' does not conform to expected input.\ncondition: {condition}\nExpected: \'before\' or \'after\'')
    # loop through the indices of 'bad' in reverse order and delete undesired entries. 
    for index in sorted(bad, reverse=True):
        del datesToSort[index]
    return datesToSort

<font face='Calibri' size='3'><b>Set some variables determining whether and how to filter the paths of the images:</b></font>

In [ ]:
eruptionDate = 20180729
condition = 'before'
filter_dates = False # if True, separate data according to details above; if anything else, run the whole dataset. 

<font face='Calibri' size='3'><b>Filter the image paths by date if filter_dates = True:</b></font>

In [ ]:
# Edit the paths to include only those that are before or after ('condition') 'eruptionDate'.
if filter_dates:
    amp_paths = sortDates(amp_paths, condition, eruptionDate)
    ingram_paths = sortDates(ingram_paths, condition, eruptionDate)
    cohr_paths = sortDates(cohr_paths, condition, eruptionDate)

print(amp_paths[-1])

<font face='Calibri'>
    <font size='5'> <b> 1. Identify Area of Interest</b> </font>
    <br>
    <font size='3'> Here we use an interactive Bokeh plot to identify our region of interest. Our region of interest must contain all of the expected deformation and a surrounding region of little to no deformation. Following our selection of this region, we will subset our data to this region. This helps reduce computation time. </font>
    </font>

<font face='Calibri'><font size='3'>Now we <b>select the bounding box for our area of interest.</b> Our area of interest should be large enough to include regions that have little to no deformation.</font></font>

In [ ]:
# Using Google maps, get rough bounding box for the area of interest
ulx = -91.34 # Upper Left X; western most longitude
lrx = -90.9 # Lower Right X; eastern most longitude
lry = -0.97 # Lower Right Y; southern most latitude
uly = -0.67 # Upper Left Y; northern most latitude

In [ ]:
pth = "ingrams/S1AA-20180712T002627-20180724T002627-DV-POEORB-12d-20x4-int-gamma/20180712T002627_20180724T002627_amp.tif"
pth2 = "ingrams/S1AA-20180712T002627-20180724T002627-DV-POEORB-12d-20x4-int-gamma/20180712T002627_20180724T002627_amp_r.tif"
!gdalwarp -overwrite $pth $pth2 -s_srs EPSG:3857 -t_srs EPSG:32606

In [ ]:
from PIL import Image
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show, output_file, gmap
import numpy as np
import gdal
import matplotlib.pylab as plt
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, GMapOptions, HoverTool

output_notebook()
output_file("tile.html")
p = gmap
tile_provider = get_provider(Vendors.STAMEN_TERRAIN)

t = "pan, wheel_zoom, tap, crosshair, reset, save, hover"

# range bounds supplied in web mercator coordinates
p = figure(title="Title", x_range=(-10167922, -10118941), y_range=(-107985, -74585),
           x_axis_type="mercator", y_axis_type="mercator", tools=t)



p.add_tools(HoverTool(tooltips = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)")
]))


p.add_tile(tile_provider)
show(p)

In [ ]:
'''
# Use an interactive Bokeh graph to select the bounding box for the subset. 
# DOES THIS AFTER YOU HAVE GIAnT WORKING!!!!
from PIL import Image
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import numpy as np
import gdal
import matplotlib.pylab as plt


output_notebook() # has to be run in order to make Bokeh display in the notebook. Otherwise, the code will run without an output. 
x_range,y_range = (ulx,lrx),(uly,lry)
#TOOLS = "pan, wheel_zoom, reset, hover, save"
x_axis_type,y_axis_type = "mercator","mercator"

im=gdal.Open(amp_paths[0])
raster_1 = im.GetRasterBand(1).ReadAsArray()
fig = plt.figure(figsize=(18,10)) # Initialize figure with a size
##########ax1 = fig.add_subplot(111) # 221 determines: 2 rows, 2 plots, first plot
###########ax1.imshow(raster_1,cmap='gray',vmin=-0.75,vmax=0.75) #,vmin=2000,vmax=10000)

#im = Image.open(tiff_paths[0])
#im = im.convert("RGBA")
#im.show()
#imarray = np.array(im)
XSize, YSize = im.RasterXSize, im.RasterYSize
print(XSize) # Number of Pixels
print(YSize) # Number of Lines

# Takes a really long time for Bokeh to load the image. I may need to downsample it somehow. 
# Bokeh also shows my map flipped. 
p1 = figure(x_range=(0,im.RasterXSize), y_range=(0,im.RasterYSize), 
            x_axis_type=x_axis_type, y_axis_type=y_axis_type,
            width=round(XSize/10), height=round(YSize/10))
p1.image_rgba(image=[raster_1], x=0, y=0, dw=XSize, dh=YSize)
show(p1)

#data = np.empty((20,20), dtype=np.uint32)
#print(f"\ndata = ",data[0])
"""
p = figure(
    title="Bounding Box Test", tools=TOOLS, x_range=x_range, y_range=y_range, 
    tooltips=[("Long (x)","$x"),("Lat (y)","$y"),("value","@image")], 
    x_axis_type=x_axis_type, y_axis_type=y_axis_type)#, 
#    source=data)

p.image(image=[img])

p.xaxis.axis_label = "Longitude"
p.yaxis.axis_label = "Latitude"

"""
#p = figure(tooltips=[("value","@image")],x_range=[0,20],y_range=[0,20])
#p.image(image=[data],x=[0],y=[0],dw=[20],dh=[20])

# must give a vector of image data for image parameter
#bk.plotting.show(p)

# plot the created object
#show(p)
'''

<font face='Calibri'>
    <font size='5'> <b> 2. Subset (Crop) Data to Area of Interest </b> </font>
    <br>
    <font size='3'> We now crop our data to our area of interest. We must do this for both the interferograms and the coherence files. </font>
    </font>

In [ ]:
# create a directory for and subset the interferograms
try:
    shutil.rmtree('ingram_subsets')
except:
    pass

!mkdir -p ingram_subsets

for ingram_path in ingram_paths:
    _, granule_name, ingram_name = ingram_path.split('/')
    
    #Using the GDAL subset service, get a small subset around the volcano
    #!wget -O ingram_subsets/{granule_name}_unw_phase.tiff "https://services.asf.alaska.edu/geospatial/subset?ulx={ulx}&lrx={lrx}&lry={lry}&uly={uly}&product={granule_name}.zip/{granule_name}/{tiff_name}"
    
    # GDAL service is out of service. Pretend that it isn't when calling the following equivalent command
    gdal_command = f"gdal_translate -epo -eco -projwin {ulx} {uly} {lrx} {lry} -projwin_srs 'WGS84' -co \"COMPRESS=DEFLATE\" -co \"TILED=YES\" -co \"COPY_SRC_OVERVIEWS=YES\" {ingram_path} ingram_subsets/{granule_name}_unw_phase.tiff > /dev/null"
    #print(f"\nCalling the command: {gdal_command}")
    !{gdal_command}
    # ' > /dev/null' sends the output of the GDAL command to a null file to suppress overly verbose output. 
    # If you suspect a problem, remove ' > /dev/null'
print("Interferograms subsetted.")
# repeat subsetting for the coherence files
for cohr_path in cohr_paths:
    _, granule_name, cohr_name = cohr_path.split('/')
    gdal_command = f"gdal_translate -epo -eco -projwin {ulx} {uly} {lrx} {lry} -projwin_srs 'WGS84' -co \"COMPRESS=DEFLATE\" -co \"TILED=YES\" -co \"COPY_SRC_OVERVIEWS=YES\" {cohr_path} ingram_subsets/{granule_name}_corr.tiff > /dev/null"
    !{gdal_command}
print("Coherence files subsetted.")
# Repeat subsetting for amplitude files for later data visualizations
for amp_path in amp_paths:
    _, granule_name, amp_name = amp_path.split('/')
    gdal_command = f"gdal_translate -epo -eco -projwin {ulx} {uly} {lrx} {lry} -projwin_srs 'WGS84' -co \"COMPRESS=DEFLATE\" -co \"TILED=YES\" -co \"COPY_SRC_OVERVIEWS=YES\" {amp_path} ingram_subsets/{granule_name}_amp.tiff > /dev/null"
    !{gdal_command}
print("Amplitude files subsetted for data visualizations.")
print("Subsetting complete.")

<font face='Calibri'><font size='4'><b>2.1 Check that the subsetted tiffs have pixels</b></font>
<br>
<font size='3'>Some of the subsetted geotiffs do not have pixels in our desired area of interest despite the -epo -eco options which should cause an error for all of these and skip them. Below, we will check which geotiffs actually have pixels in our area of interest and remove those that don't. 
<br><br>
    <b>Write a function to create a list of the geotiff paths and another to print them:</b></font>

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

<font face='Calibri'><font size='3'><b>Call get_tiff_paths() to make a list the geotiffs:</b></font>

In [ ]:
paths = f"ingram_subsets/*.tiff"  
tiff_paths = get_tiff_paths(paths)

<font face='Calibri'><font size='3'><b>Remove any empty or partial subsets:</b></font>

In [ ]:
t_path = f"{os.getcwd()}/ingram_subsets/"
remove_nan_subsets(t_path, tiff_paths)

<font face='Calibri'> <font size='4'> <b>2.2 Make sure that the subset images have the same number of pixels </b> </font>
<br>
<font size='3'>In some instances, the 'gdal_translate' function will return subsetted imagery with slightly different extents; for example, one subset may be 1000 x 1000 pixels while another is 1001 x 1000. This is usually more of a problem when different different data sensors are used as these sensors will often have different pixel sizes or their pixel locations will be slightly offset from each other. Since all of our data comes from Sentinel1, this is generally not a problem, but it is still good to double check.</font>
</font>

<font face='Calibri' size='3'><b>Update tiff_paths after possibly removing some containing incomplete data:</b></font>

In [ ]:
tiff_paths = get_tiff_paths(paths)
print(len(tiff_paths))

<font face='Calibri' size='3'><b>Check geotiffs for unique dimensions:</b></font>

In [ ]:
# Go through each '.tiff' file and get the file sizes. 
Pixels,Lines = [], []
for file in tiff_paths: 
    im = gdal.Open(file)
    raster_1 = im.GetRasterBand(1).ReadAsArray()
    XSize, YSize = im.RasterXSize, im.RasterYSize
    Pixels.append(XSize)
    Lines.append(YSize)

# get unique values
Pixel_set, Line_set = set(Pixels), set(Lines)
if len(Pixel_set) >1 or len(Line_set) > 1:
    print("Problem: More than 1 pixel or line value. This indicates two or more subsetted .tiff files have different sizes.")
else: 
    print("All subsetted .tiff files are the same size. Hurray!")
    print("Pixels, Lines = ", Pixels[0], Lines[0])

<font size='3'><b>If the '.tiff' files are different sizes, use GDAL to make them uniform.</b></font>

In [ ]:
# change value of one of the pixels to test
#Pixels[12] = Pixels[12]-10
# find the index of the smallest of the '.tiff' files. 
idx_Pixel = np.argmin(Pixels)
idx_Line =  np.argmin(Lines)
idx = max([idx_Pixel, idx_Line]) # Value 'idx' will be zero if all of the files are the same size. 

# clip the other files according to that smallest '.tiff' file. 
if idx > 0:
    PSize,LSize = Pixels[idx],Lines[idx] # Pixel and Line size all of the rasters should be. 
    for file in files:
        if file is not files[idx]:
            gdal_command = f"gdal_translate -of GTIFF -projwin {ulx} {uly} {lrx} {lry} ingram_subsets/{file} ingram_subsets/{file}"
            try: 
                !{gdal_command}
                print("Raster sizes corrected.")
            except:
                print("Drat")
else:
    print("Nothing to do.")

<font face='Calibri'>
<font size='5'> <b> 3. Prepare Data for GIAnT </b> </font>
<br>
<font size='3'> Our resultant files need to be adjusted for input into GIAnT. This involves:

- Adjusting filenames to start with the date <br>
- (Optional) Remove potentially disruptive default values <br>
- Convert data format from '.tiff' to '.flt' <br>
</font>
    
</font>

<font face='Calibri'>
    <font size='4'><b>3.1 Adjust File Names</b></font>
    <font size='3'><br>We will adjust the files to a simple format of &lt;masterdate&gt;-&lt;slavedate&gt;_&lt;unwrapped or coherence designation&gt;.tiff. This assumes that the files all come from Sentinel-1 and that every interferogram has a unique master and slave date pair. <br>This is entirely true; there are two interferograms with the same master and slave date pair. Because of this, one of these interferograms is lost during processing. To make use of every unique interferogram will require a more complex approach. To keep this exercise relatively simple, we will ignore the lost interferogram.</font> </font>

In [ ]:
tiff_dir = f"{tiff_paths[0].split('/')[0]}/"
for i in range (0, len(tiff_paths)):
    tiff_paths[i] = tiff_paths[i].split('/')[1]
print_tiff_paths(tiff_paths)   

In [ ]:
# <masterdate>-<slavedate>_<unwrapped or coherence designation>.tiff
# Rename the interferogram, coherence, and amplitude (for plotting later) files. 
# Rename files. 
for file in tiff_paths:
    if "S1" in file: # only affect those files that need to be renamed, which will have 'S1' at their start. 
        old_name, old_ext = os.path.splitext(file)
        #print(f"\nCurrent Name: {oldname}\nCurrent Extension: {oldExt}")
        master, slave = old_name[5:13], old_name[21:29]
        if "_unw" in file:
            new_name = f"{master}-{slave}_unw_phase{old_ext}"
        elif "_corr" in file:
            new_name = f"{master}-{slave}_corr{old_ext}"
        elif "_amp" in file:
            new_name = f"{master}-{slave}_amp{old_ext}"
        exists = os.path.isfile(tiff_dir+new_name)
        if exists:
            print(f"This one already exists: {new_name}")
        #print(f"New Name: {newname}")
        os.rename(tiff_dir+file, tiff_dir+new_name)

<font face='Calibri'> <font size='4'><b>3.2 Remove disruptive default values</b></font> 
    <br>
    <font size='3'>Now we can remove the potentially disruptive default values. For this lab, this step is optional. This is because the Sentinel-1 data does not have any of disruptive default values. However, for other datasources, this may be required.<br>This works by creating an entirely new geotiff with the text '_no_default' added to the file name.<br><b>IF YOU SKIP THIS STEP</b>, you will need to edit the code to exclude the use of '_no_default' when identifying which files to manipulate.</font>
</font>

In [ ]:
unw_phase_paths = glob.glob(f"{tiff_dir}*_unw_phase.tiff")
print(*unw_phase_paths, sep='\n')

In [ ]:
for file in unw_phase_paths:
    cmd = 'gdal_calc.py -A %s --outfile=%s.tiff --calc="(A>-1000)*A" --NoDataValue=0 --format=GTiff' % (file, file.replace('.tiff','_no_default'))
    os.system(cmd)
print("Disruptive default values removed.")

<font face='Calibri'> <font size='3'> Delete *_unw_phase.tiff files that have potentially disruptive default values.<br><b>SKIP THIS STEP IF YOU HAVE NOT RUN THE CODE DIRECTLY ABOVE</b>.</font> </font>

In [ ]:
for p in unw_phase_paths:
    try:
        os.remove(p)
    except:
        print(f"Error: failed to remove {p}")
if not glob.glob(f"{tiff_dir}*_unw_phase.tiff"):
    print(f"There are no *_unw_phase.tiff files containing potentially disruptive default values.")
else:
    print("WARNING: Potentially disruptive value files still present!")

<font face='Calibri'>
    <font size='4'><b>3.3 Convert data format from '.tiff' to '.flt'</b><br></font>
    <font size='3'>Now we go through the interferograms and coherence files and alter their format.<br>First, we convert the interferograms.</font>

In [ ]:
def convert_tiff_2_flt(paths):
    new_ext = '.flt'
    for file in paths: 
        new_name = file.split('.')[0]
        # create system command that will convert the file from a .tiff to a .flt
        cmd = f"gdal_translate -of ENVI {file} {new_name}{new_ext}"
        #print(f"cmd = {cmd}") # display what the command looks like. 
        try:
            # pass command to the system
            os.system(cmd)
        except: 
            print("Problem")
    print("Conversion from '.tiff' to '.flt' complete.")

In [ ]:
# Convert the unwrapped phase files from '.tiff' to '.flt'
no_default_paths = glob.glob('ingram_subsets/*unw_phase_no_default.tiff')
# Check that the number of files and the names appear correct
print(len(no_default_paths))
print(*no_default_paths, sep="\n")
# file format should appear as below
# YYYYMMDD-YYYYMMDD_unw_phase_no_default.flt

In [ ]:
convert_tiff_2_flt(no_default_paths)

<font size='3'>Now we repeat the '.tiff' to '.flt' conversion for the coherence files. </font>

In [ ]:
corr_paths = glob.glob(f"{tiff_dir}*_corr.tiff")
print(len(corr_paths))
print(*corr_paths, sep="\n")

In [ ]:
convert_tiff_2_flt(corr_paths)

In [ ]:
print(len(corr_paths))
print(*corr_paths, sep="\n")

<font face='Calibri'><font size='3'>Let's set up some path information and a date file that will be used later. </font></font>

<font face='Calibri'><font size='3'>This next step is optional.<br>Here, we can clean up our data folder by removing the subsetted geotiff files which are now duplicates. This is primarily for saving space. This step can be skipped without making modification to the code below. <br>For now, let's leave the code commented out (using the quotation marks) so we can use the subsetted geotiffs for data visualization after we use GIAnT.<br><b>If you run the code to remove all of the geotiffs, they will not be available for visualization later.</b> <i>It may be prudent to change this code to do all except the first clipped tiff.</i></font></font>

In [ ]:
# remove extra files that we no longer need (the '.tiff' files)
# Might be useful to skip this and keep the .tiff files for visualization later. 
"""
files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase_no_default'+file_ext) or f.endswith('_corr'+file_ext)] 
for file in files:
    #if '_no_default' not in file:
    try:
        os.remove(datadirectory+file)
        print(f"File Removed: {file}")
    except:
        print(f"File Not Found: {file}")
"""

<font face='Calibri'><font size='5'><b>4. Create Input Files And Code for GIAnT</b></font>
    <br>
    <font size ='3'>Let's create the input files and specialty code that GIAnT requires. These are listed below. 
        <br>
        
- ifg.list
    - List of the interferogram properties including master and slave date, perpendicular baseline, and sensor. 
- date.mli.par
    - File from which GIAnT pulls requisite information about the sensor. 
    - This is specifically for GAMMA files. When using other interferogram processing techniques, an alternate file is required. 
- prepxml_SBAS.py
    - Python function to create an xml file that specifies the processing options to GIAnT. 
    - This must be modified by the user for their particular application. 
- userfn.py
    - Python function to map the interferogram dates to a phyiscal file on disk. 
    - This must be modified by the user for their particular application. 
    </font>
    </font>

<font face='Calibri'> <font size='4'> <b>4.1 Create 'ifg.list' File </b> </font> </font>
<br>
<font size='3'> This will simple 4 column text file will communicate network information to GIAnT. It will be created within the <b>GIAnT</b> folder. 

In [ ]:
giant_dir = './GIAnT' # directory where we will perform GIAnT analysis
new_directory(giant_dir)

In [ ]:
amp_paths = glob.glob(f"{tiff_dir}*_amp.tiff")

<font face='Calibri'><font size='3'>Now we copy a clipped geotiff into our working directory for later data visualization.</font></font>

In [ ]:
shutil.copy(amp_paths[0], giant_dir)

In [ ]:
no_default_flt_paths = glob.glob(f"{tiff_dir}*_unw_phase_no_default.flt")
print(len(no_default_flt_paths))
print(*no_default_flt_paths, sep="\n")

In [ ]:
print(no_default_flt_paths[0].split('/')[1][:8])

In [ ]:
# Get all of the primary and secondary dates. 
masterdates, slavedates = [], []
for p in no_default_flt_paths:
    masterdates.append(p.split('/')[1][:8])
    slavedates.append(p.split('/')[1][9:17])
# Sort the dates according to the master dates. 
p_dates, s_dates = (list(t) for t in zip(*sorted(zip(masterdates, slavedates))))

with open( os.path.join('GIAnT', 'ifg.list'), 'w') as fid:
    for i in range(len(p_dates)):
        masterdate = p_dates[i] # pull out master Date (first set of numbers)
        slavedate = s_dates[i] # pull out slave Date (second set of numbers)
        bperp = '0.0' # according to JPL notebooks
        sensor = 'S1' # according to JPL notebooks
        fid.write(f'{masterdate}  {slavedate}  {bperp}  {sensor}\n') # write values to the 'ifg.list' file. 

<font face='Calibri'><font size='3'>You may notice that the code above sets the perpendicular baseline to a value of 0.0 m. This is not the true perpendicular baseline. That value can be found in metadata file (titled '$<$primary timestamp$>$_$<$secondary timestamp$>$.txt') that comes with the original interferogram. Generally, we would want the true baseline for each interferogram. However, since Sentinel-1 has such a short baseline, a value of 0.0 m is sufficient for our purposes. </font></font>

<font face='Calibri'> <font size='4'> <b>4.2 Create 'date.mli.par' File </b> </font> 
<br>
<font size='3'> As we are using GAMMA products, we must create a 'date.mli.par' file from which GIAnT will pull necessary information. If another processing technique is used to create the interferograms, an alterante file name and file inputs are required. </font>
</font>

In [ ]:
# Create file 'date.mli.par'
# Get WIDTH (xsize) and FILE_LENGTH (ysize) information
ds = gdal.Open(no_default_flt_paths[0], gdal.GA_ReadOnly)
n_lines = ds.RasterYSize
n_pixels = ds.RasterXSize
trans = ds.GetGeoTransform()
ds = None

# Get the center line UTC time stamp; can also be found inside <date>_<date>.txt file and hard coded
dir_name = os.listdir('ingrams')[0] # get original file name (any file can be used; the timestamps are different by a few seconds)
vals = dir_name.split('-') # break file name into parts using the separator '-'
time_stamp = vals[2][9:16] # extract the time stamp from the 2nd datetime (could be the first)
c_l_utc = int(time_stamp[0:2]) * 3600 + int(time_stamp[2:4])*60 + int(time_stamp[4:6])

radar_freq = 299792548.0 / 0.055465763 # radar frequency; speed of light divided by radar wavelength of Sentinel1 in meters

# write the 'date.mli.par' file
with open(os.path.join(giant_dir, 'date.mli.par'), 'w') as fid:
    # Method 1
    fid.write(f'radar_frequency: {radar_freq} \n') # when using GAMMA products, GIAnT requires the radar frequency. Everything else is in wavelength (m) 
    fid.write(f'center_time: {c_l_utc} \n') # Method from Tom Logan's prepGIAnT code; can also be found inside <date>_<date>.txt file and hard coded
    fid.write( 'heading: -11.9617913 \n') # inside <date>_<date>.txt file; can be hardcoded or set up so code finds it. 
    fid.write(f'azimuth_lines: {n_lines} \n') # number of lines in direction of the satellite's flight path
    fid.write(f'range_samples: {n_pixels} \n') # number of pixels in direction perpendicular to satellite's flight path

<font face='Calibri'><font size='4'><b>4.3 Make prepxml_SBAS.py</b> </font>
<br>
<font size='3'>We will create a prepxml_SBAS.py function and put it into our GIAnT working directory.</font>
</font>

<font face='Calibri'> <font size='3'><b>4.3.1 Necessary prepxml_SBAS.py edits</b></font>
<br>
<font size='3'> GIAnT comes with an example prepxml_SBAS.py, but requries significant edits for our purposes. These alterations have already been made, so we don't have to do anything now, but it is good to know the kinds of things that have to be altered. The details of some of these options can be found in the GIAnT documentation. The rest must be found in the GIAnT processing files themselves, most notably the tsxml.py and tsio.py functions. <br>The following alterations were made:
<br>
- Changed 'example' &#9658; 'date.mli.par'
- Removed 'xlim', 'ylim', 'rxlim', and 'rylim'
    - These are used for clipping the files in GIAnT. As we have already done this, it is not necessary. 
- Removed latfile='lat.map' and lonfile='lon.map'
    - These are optional inputs for the latitude and longitude maps. 
- Removed hgtfile='hgt.map'
    - This is an optional altitude file for the sensor. 
- Removed inc=21.
    - This is the optional incidence angle information. 
    - It can be a constant float value or incidence angle file. 
    - For Sentinel1, it varies from 29.1-46.0&deg;.
- Removed masktype='f4'
    - This is the mask designation. 
    - We are not using any masks for this. 
- Changed unwfmt='RMG' &#9658; unwfmt='GRD'
    - Read data using GDAL. 
- Removed demfmt='RMG'
- Changed corfmt='RMG' &#9658; corfmt='GRD'
    - Read data using GDAL. 
- Changed nvalid=30 -> nvalid=1
    - This is the minimum number of interferograms in which a pixel must be coherent. A particular pixel will be included only if its coherence is above the coherence threshold, cohth, in more than nvalid number of interferograms. 
- Removed atmos='ECMWF'
    - This is an amtospheric correction command. It depends on a library called 'pyaps' developed for GIAnT. This library has not been installed yet. 
- Changed masterdate='19920604' &#9658; masterdate='20161119'
    - Use our actual masterdate. 
    - I simply selected the earliest date as the masterdate. 

</font>

<font face='Calibri'><font size='3'>Defining a reference region is a potentially important step. This is a region at which there should be no deformation. For a volcano, this should be some significant distance away from the volcano. GIAnT has the ability to automatically select a reference region which we will use for this exercise. <b>If you wish to set your own reference region, do so in the code below and remove the '#' from before 'rxlim=[{1},...' below.</b><br>Below is an example of how the reference region would be defined. If we look at the prepxml_SBAS.py code below, rxlim and rylim, the pixel based location of the reference region, is within the code, but has been commented out. 

In [ ]:
# Define reference region
ref_region_size = [5, 5]   # Reference region size in Lines and Pixels
ref_region_center = [-0.9, -91.3] # Center of reference region in lat, lon coordinates
rxlim,rylim = [0, 10], [95, 105]

In [ ]:
m_date = min([no_default_flt_paths[i].split('/')[1][:8] for i in range(len(no_default_flt_paths))], key=int)
filtr = 1.0 / 6 # temporal filter in length of years. 

prepxml_SBAS_Template = '''
#!/usr/bin/env python
"""Example script for creating XML files for use with the SBAS processing chain. This script is supposed to be copied to the working directory and modified as needed."""


import tsinsar as ts
import argparse
import numpy as np

def parse():
    parser= argparse.ArgumentParser(description='Preparation of XML files for setting up the processing chain. Check tsinsar/tsxml.py for details on the parameters.')
    parser.parse_args()

parse()
g = ts.TSXML('data')
g.prepare_data_xml(
    'date.mli.par', proc='GAMMA', 
    #rxlim = [{1},{2}], rylim=[{3},{4}],
    inc = 21., cohth=0.10, 
    unwfmt='GRD', corfmt='GRD', chgendian='True', endianlist=['UNW','COR'])
g.writexml('data.xml')


g = ts.TSXML('params')
g.prepare_sbas_xml(nvalid=1, netramp=True, demerr=False, uwcheck=False, regu=True, masterdate='{5}', filt={6})
g.writexml('sbas.xml')


############################################################
# Program is part of GIAnT v1.0                            #
# Copyright 2012, by the California Institute of Technology#
# Contact: earthdef@gps.caltech.edu                        #
############################################################

'''

with open(os.path.join(giant_dir,'prepxml_SBAS.py'), 'w') as fid:
    fid.write(prepxml_SBAS_Template.format(giant_dir,rxlim[0],rxlim[1],rylim[0],rylim[1],m_date,filtr))

<font face='Calibri'> <font size='4'> <b>4.4 Run prepxml_SBAS.py </b> </font>
<br>
    <font size='3'> Here we run <b>prepxml_SBAS.py</b> to create the 2 needed files

- data.xml 
- sbas.xml

To use MinTS, we would run <b>prepxml_MinTS.py</b> to create

- data.xml
- mints.xml
        
These files are needed by <b>PrepIgramStack.py</b>. 
<br>
We must first switch to the GIAnT folder in which <b>prepxml_SBAS.py</b> is contained, then call it. Otherwise, <b>prepxml_SBAS.py</b> will not be able to find the file 'date.mli.par', which holds necessary processing information. 

</font> </font>

In [ ]:
# General path to the GIAnT code. This is a temporary necessity; in the future, the path to GIAnT will be unnecessary. 
giant_path = "/usr/local/GIAnT/SCR" # only for the code that we will not modify. #


#os.chdir(giant_path)
#!ls
#!cat PrepIgramStack.py

In [ ]:
# Move into your working directory
if os.getcwd() != f"{path}/GIAnT":
    os.chdir(f"{path}/GIAnT")
print(f"{os.getcwd()}:")
print(*glob.glob('*.*'), sep='\n')

In [ ]:
!python2.7 prepxml_SBAS.py

In [ ]:
!cat data.xml # display the contents of 'data.xml'

In [ ]:
!cat sbas.xml # display the contents of 'sbas.xml'

<font face='Calibri'><font size='3'>After running <b>prepxml_SBAS.py</b>, output will appear in the Juupyter Notebook in which some your input values can be checked. Make sure the two requisite xml files were produced. </font></font>

<font face='Calibri'><font size='4'><b>4.5 Create userfn.py</b></font>
<br>
<font size='3'>Before running the next piece of code, <b>PrepIgramStack.py</b>, we must create a python file called <b>userfn.py</b>. This file maps the interferogram dates to a physical file on disk. This python file must be in our working directory, <b>/GIAnT</b>. We can create this file from within the notebook using python. 

In [ ]:
aligned_dir = './ingram_subsets'
rel_dir = os.path.relpath(aligned_dir, giant_dir)
userfn_template = """
#!/usr/bin/env python
import os 

def makefnames(dates1, dates2, sensor):
    dirname = '{0}'
    root = os.path.join(dirname, dates1+'-'+dates2)
    #unwname = root+'_unw_phase.flt' # for potentially disruptive default values kept. 
    unwname = root+'_unw_phase_no_default.flt' # for potentially disruptive default values removed. 
    corname = root+'_corr.flt'
    return unwname, corname
"""

with open('userfn.py', 'w') as fid:
    fid.write(userfn_template.format(rel_dir))

<font face='Calibri'><font size='5'><b>5. Run GIAnT</b></font>
    <br>
    <font size='3'>We have now created all of the necessary files to run GIAnT. The full GIAnT process requires 3 function calls.
- PrepIgramStack.py
- ProcessStack.py
- SBASInvert.py
<br>We will make a 4th function call that is not necessary, but provides some error estimation that can be useful.
- SBASxval.py

<font face='Calibri'> <font size='4'> <b>5.1 Run PrepIgramStack.py </b> </font>
<br>
    <font size='3'> Here we run <b>PrepIgramStack.py</b> to create the files for GIAnT. This will read in the input data and the files we previously created. This will output HDF5 files. As we did not have to modify <b>PrepIgramStack.py</b>, we will call from the installed GIAnT library. <br>
Inputs:       
- ifg.list
- data.xml
- sbas.xml        

Outputs:
- RAW-STACK.h5
- PNG previews under 'Igrams' folder 
    
    </font> </font>

In [ ]:
# Display some help information
!python2.7 $giant_path/PrepIgramStack.py -h

In [ ]:
# Call PrepIgramStack.py
!python2.7 $giant_path/PrepIgramStack.py -i ifg.list
# The '-i ifg.list' is technically unnecessary as that is the default. 

In [ ]:
# check that the files really are HDF5 files as they need to be. 
#files = [f for f in os.listdir(datadirectory) if os.path.isfile(os.path.join(datadirectory,f))]
file = os.path.join('Stack','RAW-STACK.h5')
#print(files)
if not h5py.is_hdf5(file):
    print(f'Not an HDF5 file:{file}')
else:
    print("It's an HDF5 file! Hurray!")

<font face='Calibri'> <font size='4'> <b>5.2 Run ProcessStack.py </b> </font>
<br>
    <font size='3'> This seems to be an optional step. Does atmospheric corrections and estimation of orbit residuals. <br>
Inputs:

- HDF5 files from PrepIgramStack.py, RAW-STACK.h5
- data.xml 
- sbas.xml
- GPS Data (optional; we don't have this)
- Weather models (downloaded automatically)

Outputs: 

- HDF5 files, PROC-STACK.h5
        
These files are then fed into SBAS. 
</font> </font>

In [ ]:
# Display some help information
!python2.7 $giant_path/ProcessStack.py -h

In [ ]:
!python2.7 $giant_path/ProcessStack.py

In [ ]:
file = os.path.join('Stack','PROC-STACK.h5')
#print(files)
if not h5py.is_hdf5(file):
    print(f'Not an HDF5 file:{file}')
else:
    print("It's an HDF5 file! Hurray!")

<font face='Calibri'> <font size='4'> <b>5.3 Run SBASInvert.py </b> </font>
<br>
    <font size='3'> Actually do the time series. 
 
Inputs

- HDF5 file, PROC-STACK.h5
- data.xml
- sbas.xml

Outputs

- HDF5 file: LS-PARAMS.h5

</font> </font>


In [ ]:
# Display some help information
!python2.7 $giant_path/SBASInvert.py -h

In [ ]:
!python2.7 $giant_path/SBASInvert.py

<font face='Calibri'> <font size='4'> <b>5.4 Run SBASxval.py </b> </font>
<br>
    <font size='3'> Get an uncertainty estimate for each pixel and epoch using a Jacknife test. 
 
Inputs: 

- HDF5 files, PROC-STACK.h5
- data.xml
- sbas.xml

Outputs:

- HDF5 file, LS-xval.h5

</font> </font>

In [ ]:
# Display some help information
!python2.7 $giant_path/SBASxval.py -h

In [ ]:
#!python2.7 $giant_path/SBASxval.py

<font face='Calibri'><font size='5'><b>6. Data Visualization</b></font>
<br>
    <font size='3'>Now we visualize the data. This is largely copied from Lab4B. </font></font>



<font face='Calibri'><font size='3'>First, we have to load the stack produced by GIAnT and read it into an array so we can manipulate and display it.</font></font>

In [ ]:
filename = 'Stack/LS-PARAMS.h5'
f = h5py.File(filename,'r')
# List all groups
print("Keys: %s" %f.keys())

<font face='Calibri'><font size='3'>Details on what each of these keys means can be found in the GIAnT documentation. For now, the only keys with which we are concerned are 'recons' (the filtered time series of each pixel) and 'dates' (the dates of acquisition). It is important to note that the dates are given in a type of Julian Day number called Rata Die number. This will have to be converted later, but this can easily be done via one of several different methods in Python. </font></font>

In [ ]:
# Get our data from the stack
data_cube = f[('recons')]
# Get the dates for each raster from the stack
dates = list(f['dates']) # these dates appear to be given in Rata Die style: floor(Julian Day Number - 1721424.5). 
if data_cube.shape[0] is not len(dates):
    print('Problem')
    print('Number of rasters in data_cube: ', data_cube.shape[0])
    print('Number of dates: ', len(dates))

<font face='Calibri'><font size='3'>Display and save an amplitude image of the volcano. </font></font>

In [ ]:
# Plot amplitude image with transparency determined by alpha. 
plt.rcParams.update({'font.size': 14})
radar_tiff = '20180724-20180805_amp.tiff'
radar=gdal.Open(radar_tiff)
im_radar = radar.GetRasterBand(1).ReadAsArray()

<font face='Calibri' size='3'><b>Remove zeros is numpy array to avoid divide-by-zero errors when calling np.log10():</b> </font>

In [ ]:
# numpy requires use of a range-based for loop
for i in range (0, len(im_radar)):
    for x in range (0, len(im_radar[i])):
        if im_radar[i][x] == 0.:
            im_radar[i][x] = 0.0000001

In [ ]:
dbplot = np.log10(im_radar)
vmin=np.percentile(dbplot,3)
vmax=np.percentile(dbplot,97)
fig = plt.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(111) # 221 determines: 2 rows, 2 plots, first plot
ax1.imshow(dbplot, cmap='gray',vmin=vmin,vmax=vmax,alpha=1);
plt.title('Example dB-scaled SAR Image for Ifgrm 20161119-20170106')
plt.grid()
plt.savefig('SierraNegra-dBScaled-AmplitudeImage.png',dpi=200,transparent='false')

<font face='Calibri'><font size='3'>Display an overlay of the clipped deformation map and amplitude image. </font></font>

In [ ]:
# We will define a short function that can plot an overaly of our radar image and deformation map. 
def defNradar_plot(deformation,radar):
    fig = plt.figure(figsize=(18,10))
    ax = fig.add_subplot(111)
    vmin=np.percentile(radar,3)
    vmax=np.percentile(radar,97)
    ax.imshow(radar,cmap='gray',vmin=vmin,vmax=vmax)
    fin_plot = ax.imshow(deformation,cmap='RdBu',vmin=-50.0,vmax=50.0,alpha=0.75)
    fig.colorbar(fin_plot,fraction=0.24,pad=0.02)
    ax.set(title="Integrated Defo [mm] Overlain on Clipped db-Scaled Amplitude Image")
    plt.grid()
    return None
# Get deformation map and radar image we wish to plot
deformation = data_cube[data_cube.shape[0]-1]
# Call function to plot an overlay of our deformation map and radar image.
defNradar_plot(deformation,dbplot)
plt.savefig('SierraNegra-DeformationComposite.png',dpi=200,transparent='false')

<font face='Calibri'><font size='3'>Create an animation of the deformation</font></font>

In [ ]:
# Convert from Rata Die number (similar to Julian Day number) contained in 'dates' to Gregorian date. 
tindex = []
for d in dates:
    tindex.append(date.fromordinal(int(d)))

In [ ]:
%%capture
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(data_cube.flatten(),5)
vmax=np.percentile(data_cube.flatten(),95)


im = ax.imshow(data_cube[0],cmap='RdBu',vmin=-50.0,vmax=50.0)
ax.set_title("Animation of Deformation Time Series - Sierra Negra, Galapagos")
fig.colorbar(im)
plt.grid()

def animate(i):
    ax.set_title("Date: {}".format(tindex[i]))
    im.set_data(data_cube[i])
    
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=data_cube.shape[0], interval=400)

In [ ]:
rc('animation', embed_limit=10.0**9)

In [ ]:
# Display the animation
HTML(ani.to_jshtml())

In [ ]:
# Save the animation as a 'gif' file. 
ani.save('SierraNegraDeformationTS_filt={0}.gif'.format(filtr), writer='pillow', fps=2)

<font face='Calibri'><font size='5'><b>7. Alter the time filter parameter</b></font><br>
    <font size='3'>Looking at the video above, you may notice that the deformation has a very smoothed appearance. This may be because of our time filter which is currently set to 1 year ('filt=1.0' in the prepxml_SBAS.py code). Let's repeat the lab from there with 2 different time filters. <br>First, using no time filter ('filt=0.0') and then using a 1 month time filter ('filt=0.082'). Change the output file name for anything you want saved (e.g., 'SierraNegraDeformationTS.gif' to 'YourDesiredFileName.gif'). Otherwise, it will be overwritten. <br><br>How did these changes affect the output time series?<br>How might we figure out the right filter length?<br>What does this say about the parameters we select? </font></font>

<font face='Calibri'><font size='5'><b>8. Clear data from the Notebook (optional)</b></font>
    <br>
    <font size='3'>This lab has produced a large quantity of data. If you look at this notebook in your home directory, it should now be ~80 MB. This can take a long time to load in a Jupyter Notebook. It may be useful to clear the cell outputs, which will restore the Notebook to its original size. <br>To clear the cell outputs, go Cell->All Output->Clear. This will clear the outputs of the Jupyter Notebook and restore it to its original size of ~60 kB. This will not delete any of the files we have created. </font>
    </font>

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - Feb 2019 </i>
</font>